In [ ]:
import spacy
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3

import requests
from bs4 import BeautifulSoup
import spacy

def traitement_document_gallica(url):
    try:
        # Récupération du contenu HTML du document
        response = requests.get(url)
        text = response.text
        
        # Suppression des balises HTML
        soup = BeautifulSoup(text, "html.parser")
        clean_text = soup.get_text()
        
        # Chargement du modèle de langue française
        nlp = spacy.load("fr_core_news_sm")
        
        # Traitement NER
        doc = nlp(clean_text)
        
        # Affichage des entités nommées détectées
        for ent in doc.ents:
            print(f"{ent.text}: {ent.label_} ({spacy.explain(ent.label_)} ) ({spacy.displacy.render(doc, style='ent')})")
        
        return doc  # Vous pouvez également retourner le doc ou d'autres données selon vos besoins
        
    except requests.RequestException as e:
        print("Erreur lors de la requête :", str(e))
        return None

# Appel de la méthode pour traiter le document Gallica avec l'URL spécifié
url_gallica = "https://gallica.bnf.fr/ark:/12148/bpt6k1426047z"
traitement_document_gallica(url_gallica)

In [ ]:
# fonction pour normaliser les entités nommées détectées dans un document Gallica
def normaliser_entites_nommees(url):
    try:
        # Récupération du contenu HTML du document
        response = requests.get(url)
        text = response.text
        
        # Suppression des balises HTML
        soup = BeautifulSoup(text, "html.parser")
        clean_text = soup.get_text()
        
        # Chargement du modèle de langue française
        nlp = spacy.load("fr_core_news_sm")
        
        # Traitement NER
        doc = nlp(clean_text)
        
        # Normalisation des entités nommées détectées
        entites_normalisees = []
        for ent in doc.ents:
            if ent.label_ == "LOC":  # Exemple : normalisation des lieux
                entite_normalisee = normaliser_lieux(ent.text)
                entites_normalisees.append((entite_normalisee, ent.label_))
            elif ent.label_ == "DATE":  # Exemple : normalisation des dates
                entite_normalisee = normaliser_dates(ent.text)
                entites_normalisees.append((entite_normalisee, ent.label_))
            elif ent.label_ == "PER":  # Exemple : normalisation des personnes
                entite_normalisee = normaliser_personnes(ent.text)
                entites_normalisees.append((entite_normalisee, ent.label_))    
            else:
                entites_normalisees.append((ent.text, ent.label_))
        
        return entites_normalisees
        
    except requests.RequestException as e:
        print("Erreur lors de la requête :", str(e))
        return None

# Fonctions de normalisation des lieux
def normaliser_lieux(entite):
    # Par exemple, convertir en majuscules, supprimer les espaces, etc.
    return entite.upper().replace("  ", " ")

# Fonctions de normalisation des dates
def normaliser_dates(entite):
   return entite
def normaliser_personnes(entite):
    # convertir en majuscules, supprimer les espaces en trop, etc.
    return entite.upper().replace("  ", " ")
    
# Appel de la méthode de normalisation des entités nommées pour l'URL spécifiée
url_gallica = "https://gallica.bnf.fr/ark:/12148/bpt6k1426047z"
entites_normalisees = normaliser_entites_nommees(url_gallica)

# Affichage des entités nommées normalisées
if entites_normalisees:
    print("Entités nommées normalisées:")
    for entite, label in entites_normalisees:
        print(f"{entite}: {label}")

In [4]:
# fonction pour inserer les entites nommées dans une base de données sqlite à parir la liste des entités nommées normalisées
def inserer_entites_nommees(entites_normalisees):
    try:
        # Connexion à la base de données SQLite
        conn = sqlite3.connect("entites.db")
        cursor = conn.cursor()
        
        # Création de la table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS entites_nommees (
                id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
                entite TEXT,
                label TEXT
            )
        """)
        
        # Insertion des entités nommées dans la base de données
        cursor.executemany("""
            INSERT INTO entites_nommees (entite, label) VALUES (?, ?)
        """, entites_normalisees)
        
        # Sauvegarde de la transaction
        conn.commit()
        
        # Fermeture de la connexion à la base de données
        conn.close()
        
    except sqlite3.Error as e:
        print("Erreur lors de la requête SQL :", str(e))
        return None
# Appel de la méthode pour insérer les entités nommées dans la base de données
inserer_entites_nommees(entites_normalisees) 

In [35]:
# stocker les entités nommées normalisées dans un DataFrame
df = pd.DataFrame(entites_normalisees, columns=["entite", "label"])
df.head()
# exporter le DataFrame au format CSV
entite_csv = df.to_csv("entites_normalisees.csv", index=False)

In [40]:
# stocker les entités nommées normalisées dans une base de données SQLite sans utiliser une fonction
conn = sqlite3.connect("entites_normalisees1.db")
df.to_sql("entites_normalisees", conn, if_exists="replace", index=False)
conn.close()

In [9]:
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import requests
from bs4 import BeautifulSoup
import spacy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


def preparer_donnees_classification(url):
    try:
        # Récupération du contenu HTML du document
        response = requests.get(url)
        text = response.text
        
        # Suppression des balises HTML
        soup = BeautifulSoup(text, "html.parser")
        clean_text = soup.get_text()
        
        # Chargement du modèle de langue française
        nlp = spacy.load("fr_core_news_sm")
        
        # Traitement NER
        doc = nlp(clean_text)

        data = []
        for ent in doc.ents:
            if ent.label_ in ['PER', 'ORG']:  # Filtrer les entités de type Personne (PER) ou Organisation (ORG)
                data.append((ent.text, ent.label_))

        df = pd.DataFrame(data, columns=['Texte', 'Label'])
        df['Label'] = df['Label'].map({'PER': 0, 'ORG': 1})

        X_train, X_test, y_train, y_test = train_test_split(df['Texte'], df['Label'], test_size=0.2, random_state=42)

        vectorizer = TfidfVectorizer()
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        model = LogisticRegression(max_iter=1000)
        model.fit(X_train_tfidf, y_train)

        predictions = model.predict(X_test_tfidf)

        accuracy = accuracy_score(y_test, predictions)
        print(f'Accuracy: {accuracy}')

        return model, vectorizer
    
    except requests.RequestException as e:
        print("Erreur lors de la requête :", str(e))
        return None

# Utilisation de l'URL url_gallica dans la fonction de préparation des données pour la classification
url_gallica = "https://gallica.bnf.fr/ark:/12148/bpt6k1426047z"
model, vectorizer = preparer_donnees_classification(url_gallica)


Accuracy: 0.6666666666666666


In [8]:
# Utilisation du modèle pour la classification
"""
Dans cet exemple, la fonction classifier_entites_nommees prend le modèle entraîné et le vectorizer
 (qui ont été obtenus à partir de la préparation des données) ainsi qu'un texte à classifier. 
 Cette fonction vectorise le texte en utilisant le vectorizer, puis utilise le modèle pour prédire 
 la catégorie de l'entité dans le texte donné.
"""
def classifier_entites_nommees(model, vectorizer, texte):
    # Vectorisation du texte avec le vectorizer
    texte_vectorise = vectorizer.transform([texte])
    
    # Prédiction de la catégorie de l'entité
    prediction = model.predict(texte_vectorise)
    
    # Interprétation de la prédiction
    if prediction[0] == 0:
        return "Personne"
    elif prediction[0] == 1:
        return "Organisation"
    else:
        return "Catégorie indéterminée"

# Texte à classifier
texte_a_classifier = "Texte à classer, extrait du document Gallica..."

# Utilisation du modèle pour classifier le texte
categorie = classifier_entites_nommees(model, vectorizer, texte_a_classifier)
print(f"Catégorie de l'entité : {categorie}")


Catégorie de l'entité : Catégorie indéterminée


Ici, on compare les résultats de SpaCy et Flair pour le français sur le même texte
pour les entités de type Personne (PER) et Organisation (ORG).
Cela utilise la bibliothèque Flair pour extraire les entités nommées du texte, puis les compare avec les entités extraites par SpaCy pour permettre une comparaison visuelle. Vous pouvez ajuster les paramètres selon vos besoins spécifiques.

In [ ]:
import spacy
from bs4 import BeautifulSoup
import requests
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger

# URL du document sur Gallica
url = "https://gallica.bnf.fr/ark:/12148/bpt6k1426047z"
response = requests.get(url)
text = response.text

# Supprimer les balises HTML
soup = BeautifulSoup(text, "html.parser")
clean_text = soup.get_text()

# Modèle de langue française avec réseaux neuronaux de SpaCy
nlp_spacy = spacy.load("fr_core_news_sm")

# Traitement NER avec SpaCy
doc_spacy = nlp_spacy(clean_text)
entities_spacy = [(ent.text, ent.label_) for ent in doc_spacy.ents if ent.label_ in ['PER', 'ORG']]

# Traitement NER avec Flair
tagger = SequenceTagger.load('ner-ontonotes')
sentence = Sentence(clean_text)
tagger.predict(sentence)
entities_flair = [(entity.text, entity.get_labels()[0].value) for entity in sentence.get_spans('ner')]

# Stockage des entités nommées dans des DataFrames
df_spacy = pd.DataFrame(entities_spacy, columns=['Entité', 'Type (SpaCy)'])
df_flair = pd.DataFrame(entities_flair, columns=['Entité', 'Type (Flair)'])

# Affichage des DataFrames pour comparaison
print("Résultats de SpaCy:")
print(df_spacy.head())  # Affiche les 5 premières entités de SpaCy
print("\nRésultats de Flair:")
print(df_flair.head())  # Affiche les 5 premières entités de Flair
